In [1]:
import os

from pathling import PathlingContext
from pyspark.sql.functions import lit

#MIMIC_FHIR_PATH="../_data/mimic4-demo-ptl"
MIMIC_FHIR_PATH="/Users/szu004/datasets/work/mimic-iv/mimic4-ptl"

In [2]:
os.environ['SPARK_CONF_DIR'] = os.path.abspath('../env/spark-conf')

In [3]:
pc = PathlingContext.create()

:: loading settings :: url = jar:file:/Users/szu004/miniconda3/envs/sof_mimic/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/szu004/.ivy2/cache
The jars for the packages stored in: /Users/szu004/.ivy2/jars
au.csiro.pathling#library-runtime added as a dependency
io.delta#delta-spark_2.12 added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-e4bde3f4-bdd9-4d31-827c-ac3defca7a59;1.0
	confs: [default]
	found au.csiro.pathling#library-runtime;7.1.0-SNAPSHOT in local-m2-cache
	found io.delta#delta-spark_2.12;3.2.0 in local-m2-cache
	found io.delta#delta-storage;3.2.0 in local-m2-cache
	found org.antlr#antlr4-runtime;4.9.3 in local-m2-cache
	found org.apache.hadoop#hadoop-aws;3.3.4 in local-m2-cache
	found com.amazonaws#aws-java-sdk-bundle;1.12.262 in local-m2-cache
	found org.wildfly.openssl#wildfly-openssl;1.0.7.Final in local-m2-cache
:: resolution report :: resolve 134ms :: artifacts dl 6ms
	:: modules in use:
	au.csiro.pathling#library-runtime;7.1.0-SNAPSHOT from local-m2-cache in [default]
	co

2024-09-09 11:12:14 WARN  NativeCodeLoader:60 - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2024-09-09 11:12:16 INFO  Version:21 - HV000001: Hibernate Validator null
2024-09-09 11:12:16 DEBUG AbstractConfigurationImpl:176 - Setting custom MessageInterpolator of type au.csiro.pathling.shaded.org.hibernate.validator.messageinterpolation.ParameterMessageInterpolator
2024-09-09 11:12:16 DEBUG ValidationXmlParser:122 - Trying to load META-INF/validation.xml for XML based Validator configuration.
2024-09-09 11:12:16 DEBUG ResourceLoaderHelper:61 - Trying to load META-INF/validation.xml via TCCL
2024-09-09 11:12:16 DEBUG ResourceLoaderHelper:67 - Trying to load META-INF/validation.xml via Hibernate Validator's class loader
2024-09-09 11:12:16 DEBUG ValidationXmlParser:129 - No META-INF/validation.xml found. Using annotation based configuration only.
2024-09-09 11:12:16 DEBUG TraversableResolvers:61 - Cannot find jakarta.persistence.Per

In [4]:
spark = pc.spark

In [5]:
print('spark.sql.ansi.enabled', spark.conf.get("spark.sql.ansi.enabled"))
print('spark.sql.timestampType', spark.conf.get("spark.sql.timestampType"))

spark.sql.ansi.enabled false
spark.sql.timestampType TIMESTAMP_LTZ


In [6]:
data = pc.read.parquet(MIMIC_FHIR_PATH)
data.read('Patient').show()

2024-09-09 11:12:26 WARN  SparkStringUtils:72 - Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+--------------------+--------------------+--------------------+-------------+--------+----+--------------------+------+--------------------+-------+------+----------+---------------+----------------+-------+--------------------+--------------------+--------------------+-----+-------+--------------------+-------------------+--------------------+----+----------+--------------------+
|                  id|        id_versioned|                meta|implicitRules|language|text|          identifier|active|                name|telecom|gender| birthDate|deceasedBoolean|deceasedDateTime|address|       maritalStatus|multipleBirthBoolean|multipleBirthInteger|photo|contact|       communication|generalPractitioner|managingOrganization|link|      _fid|          _extension|
+--------------------+--------------------+--------------------+-------------+--------+----+--------------------+------+--------------------+-------+------+----------+---------------+----------------+-------+--------------------+-

## icustay_detail

### Patient

In [7]:
rv_patient = data.view('Patient', 
    select = [
        {
            "column": [
                {
                    "name": "patient_id",
                    "path": "getResourceKey()",
                    "collection": False
                },
                {
                    "name": "gender",
                    "path": "gender",
                    "collection": False
                },
                {
                    "name": "race_system",
                    "path": "extension('http://hl7.org/fhir/us/core/StructureDefinition/us-core-race').extension('ombCategory').value.ofType(Coding).system",
                    "collection": False
                },
                {
                    "name": "race_code",
                    "path": "extension('http://hl7.org/fhir/us/core/StructureDefinition/us-core-race').extension('ombCategory').value.ofType(Coding).code",
                    "collection": False
                },
                {
                    "name": "ethnicity_system",
                    "path": "extension('http://hl7.org/fhir/us/core/StructureDefinition/us-core-ethnicity').extension('ombCategory').value.ofType(Coding).system",
                    "collection": False
                },
                {
                    "name": "ethnicity_code",
                    "path": "extension('http://hl7.org/fhir/us/core/StructureDefinition/us-core-race').extension('ombCategory').value.ofType(Coding).code",
                    "collection": False
                },
            ]
        }
    ]
)

rv_patient.createOrReplaceTempView('rv_patient')
rv_patient.toPandas()

,patient_id,gender,race_system,race_code,ethnicity_system,ethnicity_code
0,Patient/c9c2b8c3-f36e-570f-91e6-17880a4cfbdb,female,None,None,None,None
1,Patient/cc409b2b-2c36-5232-8233-5bd5b54c1167,male,urn:oid:2.16.840.1.113883.6.238,2106-3,urn:oid:2.16.840.1.113883.6.238,2106-3
2,Patient/b1cea26f-6b79-59e0-a825-0837f3d0c065,male,http://terminology.hl7.org/CodeSystem/v3-NullF...,OTH,urn:oid:2.16.840.1.113883.6.238,OTH
3,Patient/4e5e8c04-28b6-5a10-9533-0842999a2fa1,male,None,None,None,None
4,Patient/fe8673eb-f216-550f-a007-f9a4bec301ab,female,urn:oid:2.16.840.1.113883.6.238,2028-9,urn:oid:2.16.840.1.113883.6.238,2028-9
...,...,...,...,...,...,...
299707,Patient/6e090777-50f8-5e99-be1b-b364da8ac404,female,urn:oid:2.16.840.1.113883.6.238,2106-3,urn:oid:2.16.840.1.113883.6.238,2106-3
299708,Patient/2ab4cc77-3fdf-5e37-8abe-190550020467,female,urn:oid:2.16.840.1.113883.6.238,2106-3,urn:oid:2.16.840.1.113883.6.238,2106-3
299709,Patient/26b6e07c-9923-5a1c-bc78-babbf7d89f5c,female,urn:oid:2.16.840.1.113883.6.238,2106-3,urn:oid:2.16.840.1.113883.6.238,2106-3
299710,Patient/3ccb0d94-f544-592c-93d4-2e15ad4e7917,male,urn:oid:2.16.840.1.113883.6.238,2106-3,urn:oid:2.16.840.1.113883.6.238,2106-3


In [10]:
spark.sql("""
CREATE OR REPLACE TEMP VIEW dv_patient AS
SELECT patient_id AS subject_id,
    CASE 
        WHEN gender = 'male' THEN 'M'
        WHEN gender = 'female' THEN 'F'
    END AS gender,
    CASE 
        WHEN race_system = 'urn:oid:2.16.840.1.113883.6.238' THEN
            CASE 
                WHEN race_code = '2106-3' THEN 
                    CASE
                        WHEN ethnicity_system = 'urn:oid:2.16.840.1.113883.6.238' AND ethnicity_code = '2135-2' THEN 'HISPANIC' 
                        ELSE 'WHITE'
                    END
                WHEN race_code = '2054-5' THEN 'BLACK'
                WHEN race_code = '2028-9' THEN 'ASIAN'
            END
    END AS race
FROM rv_patient;    
""")
dv_patient = spark.sql("SELECT * FROM dv_patient")
dv_patient.toPandas()

,subject_id,gender,race
0,Patient/c9c2b8c3-f36e-570f-91e6-17880a4cfbdb,F,None
1,Patient/cc409b2b-2c36-5232-8233-5bd5b54c1167,M,WHITE
2,Patient/b1cea26f-6b79-59e0-a825-0837f3d0c065,M,None
3,Patient/4e5e8c04-28b6-5a10-9533-0842999a2fa1,M,None
4,Patient/fe8673eb-f216-550f-a007-f9a4bec301ab,F,ASIAN
...,...,...,...
299707,Patient/6e090777-50f8-5e99-be1b-b364da8ac404,F,WHITE
299708,Patient/2ab4cc77-3fdf-5e37-8abe-190550020467,F,WHITE
299709,Patient/26b6e07c-9923-5a1c-bc78-babbf7d89f5c,F,WHITE
299710,Patient/3ccb0d94-f544-592c-93d4-2e15ad4e7917,M,WHITE


### Encounter(ICU) 


In [9]:
rv_icu_encounter = data.view('Encounter',
                    select = [
                        {
                            "column": [
                                {
                                    "name": "encounter_id",
                                    "path": "getResourceKey()",
                                    "type": "string",
                                    "collection": False
                                },
                                {
                                    "name": "patient_id",
                                    "path": "subject.getReferenceKey()",
                                    "type": "string",
                                    "collection": False
                                },
                                {
                                    "name": "admit_date",
                                    "path": "period.start",
                                    "type": "dateTime",
                                    "collection": False
                                },
                                {
                                    "name": "disch_date",
                                    "path": "period.end",
                                    "type": "dateTime",
                                    "collection": False
                                }

                            ]
                        }
                    ],
                    where= [
                        {
                            "path":"class.system = 'http://terminology.hl7.org/CodeSystem/v3-ActCode'"
                        },
                        {
                            "path":"class.code = 'ACUTE'"
                        },

                    ]
                    )

rv_icu_encounter.createOrReplaceTempView('rv_icu_encounter')
rv_icu_encounter.toPandas()

,encounter_id,patient_id,admit_date,disch_date
0,Encounter/18f74cab-e07d-5ca6-98b8-01f8f9956caa,Patient/0a8eebfd-a352-522e-89f0-1d4a13abdebc,2180-07-23T14:00:00-04:00,2180-07-23T23:50:47-04:00
1,Encounter/c8376997-bae8-5173-b8ab-1a685a4494fd,Patient/b9751acd-fc69-520d-b2e6-f7e232dbea03,2189-06-27T08:42:00-04:00,2189-06-27T20:38:27-04:00
2,Encounter/3cf8d7df-175b-5367-b816-c2bcdb9cb045,Patient/a6e7e991-6801-5425-b435-4ca6b7decfcc,2157-11-20T19:18:02-05:00,2157-11-21T22:08:00-05:00
3,Encounter/08914785-89e0-5619-b89c-a1d1dd27c2d7,Patient/a6e7e991-6801-5425-b435-4ca6b7decfcc,2157-12-19T15:42:24-05:00,2157-12-20T14:27:41-05:00
4,Encounter/5b30c499-1078-550d-a221-ba9053daf599,Patient/4c9069e4-486b-53a6-8a49-b8520d49ef7e,2110-04-11T15:52:22-04:00,2110-04-12T23:59:56-04:00
...,...,...,...,...
73176,Encounter/100ae19c-ceec-5efd-a374-5ea982dbc75a,Patient/87f081e2-6d7f-5a3a-9920-1603b850c0be,2148-11-19T14:23:43-05:00,2148-11-26T13:12:15-05:00
73177,Encounter/009cf2fb-a973-5cd5-81e0-5defcb5d4a9e,Patient/69155ccf-53fc-514f-9773-c16e388a4f77,2139-10-10T19:18:00-04:00,2139-10-11T18:21:28-04:00
73178,Encounter/fd85d65c-212e-5a9d-a9d8-5d4169680efe,Patient/3f9b77b4-f175-5fe5-a00d-f1f87e11e49b,2149-01-08T18:12:00-05:00,2149-01-10T13:11:02-05:00
73179,Encounter/28528404-7775-5a28-b9a9-114e7696be3a,Patient/c37de21a-7f0e-5131-9558-3acb186fe953,2164-09-12T09:26:28-04:00,2164-09-17T16:35:15-04:00


In [10]:
dv_icu_encounter= spark.sql("""
WITH tv AS (
    SELECT 
        patient_id AS subject_id,
        encounter_id AS stay_id,
        CAST(admit_date AS TIMESTAMP) icu_intime, 
        CAST(disch_date  AS TIMESTAMP) icu_outtime 
    FROM rv_icu_encounter)
SELECT *, 
    row_number() over (PARTITION BY subject_id, stay_id ORDER BY icu_intime ASC) AS icustay_seq 
FROM tv
""")
dv_icu_encounter.createOrReplaceTempView('dv_icu_encounter')
dv_icu_encounter.toPandas()

,subject_id,stay_id,icu_intime,icu_outtime,icustay_seq
0,Patient/0009f3e1-31aa-5bed-8260-25b60d581bcb,Encounter/586b0364-50a7-595f-912d-ded5e501bded,2178-11-22 13:27:27,2178-11-23 14:32:24,1
1,Patient/0009f3e1-31aa-5bed-8260-25b60d581bcb,Encounter/935de6db-a70a-5fc5-b3e8-b5a7b2fd0900,2178-06-10 01:12:13,2178-06-14 07:23:13,1
2,Patient/0009f4ea-2d4c-5bfe-8446-614cef559bde,Encounter/fa511caa-1386-54c2-bbbf-badfd042c50a,2193-03-28 17:27:39,2193-03-29 04:16:55,1
3,Patient/00120424-417b-501d-9c25-9f1ab3e92c0b,Encounter/8261e220-e913-5bf8-9bc5-be31f407ced3,2115-10-26 10:16:00,2115-10-27 07:52:17,1
4,Patient/00233b20-c851-5075-9f95-8125187ce6bc,Encounter/902e3d68-7f4a-56c5-9e84-dfc319fd1104,2115-08-15 16:10:49,2115-09-25 22:34:12,1
...,...,...,...,...,...
73176,Patient/ffecb2bb-f6d4-586e-b06f-06388c168290,Encounter/dd6b4730-44ba-587d-bc6f-b67df694c2d0,2196-05-14 00:15:00,2196-05-14 12:27:55,1
73177,Patient/ffed48b6-dbd7-581a-81dd-54f180968dfc,Encounter/150a540c-ba8c-512b-83e9-d9d412434213,2130-08-02 12:42:00,2130-08-03 05:46:24,1
73178,Patient/ffeedba7-07f4-55b2-9adb-2005ba6ee162,Encounter/14ac687a-2dcc-5efe-8a28-788ad509bfb6,2133-02-09 16:12:00,2133-02-10 15:07:42,1
73179,Patient/fff48550-dd01-5b11-9542-15660bdd6b17,Encounter/57bed9f7-9fd6-5a1c-9a31-90f42acdb6c2,2150-09-09 10:31:00,2150-09-11 06:20:23,1


### ICU Stay Detail

In [11]:
icustay_detail = spark.sql("""
SELECT 
    enc.subject_id, enc.stay_id, pat.gender, pat. race, enc.icu_intime, enc.icu_outtime, enc.icustay_seq 
FROM dv_icu_encounter enc
LEFT OUTER JOIN dv_patient pat ON enc.subject_id = pat.subject_id
""")
icustay_detail.toPandas()
icustay_detail.write.parquet('../_data/mimic_derrived/icustay_detail.parquet', mode='overwrite')

## Oxygen Delivery Device

In [12]:
CS_MIMIC_CHARTEVENTS_D_ITEMS = 'http://mimic.mit.edu/fhir/mimic/CodeSystem/mimic-chartevents-d-items'
CS_MIMIC_D_LABITEMS = 'http://mimic.mit.edu/fhir/mimic/CodeSystem/mimic-d-labitems'

def obs_str_for_coding(system, code, view_name = None):
    obs_view = data.view('Observation',
                         select = [
                             {
                                 "column": [
                                     {
                                         "name": "id",
                                         "path": "id",
                                         "type": "id",
                                         "collection": False
                                     },
                                     {
                                         "name": "patient_id",
                                         "path": "subject.getReferenceKey()",
                                         "type": "string",
                                         "collection": False
                                     },
                                     {
                                         "name": "encounter_id",
                                         "path": "encounter.getReferenceKey()",
                                         "type": "string",
                                         "collection": False
                                     },
                                     {
                                         "name": "charttime",
                                         "path": "effectiveDateTime",
                                         "type": "dateTime",
                                         "collection": False
                                     },
                                     {
                                         "name": "storetime",
                                         "path": "issued",
                                         "type": "instant",
                                         "collection": False
                                     },
                                     {
                                         "name": "value",
                                         "path": "valueString",
                                         "type": "string",
                                         "collection": False
                                     }
                                 ]
                             }
                         ],
                         where= [
                             {
                                 "path":f"code.coding.system = '{system}'"
                             },
                             {
                                 "path":f"code.coding.code = '{code}'"
                             },

                         ]
                         )
    if view_name is not None:
        obs_view.createOrReplaceTempView(view_name)
    return obs_view

rv_oxygen_delivery_device = obs_str_for_coding(CS_MIMIC_CHARTEVENTS_D_ITEMS, '226732', 'rv_oxygen_delivery_device')
rv_oxygen_delivery_device.toPandas()

2024-09-09 10:25:09 WARN  GarbageCollectionMetrics:72 - To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


,id,patient_id,encounter_id,charttime,storetime,value
0,5e09a5db-9e2e-588c-9656-06ff32c8bd55,Patient/327b0c02-5e31-5c1d-979b-3b3b942b5770,Encounter/e49adf4e-9a22-5063-8bda-0c464c3e3261,2156-03-24T16:00:00-04:00,2156-03-25 05:52:00,T-piece
1,d5b4046c-0ac6-534e-89e5-88606f0441d4,Patient/327b0c02-5e31-5c1d-979b-3b3b942b5770,Encounter/e49adf4e-9a22-5063-8bda-0c464c3e3261,2156-03-24T10:00:00-04:00,2156-03-24 23:57:00,T-piece
2,67da8faa-0fab-53d2-aa23-53a18dd40118,Patient/327b0c02-5e31-5c1d-979b-3b3b942b5770,Encounter/e49adf4e-9a22-5063-8bda-0c464c3e3261,2156-03-24T04:00:00-04:00,2156-03-24 18:59:00,T-piece
3,ec11b00a-24d3-53a9-a75c-a5eebb36a265,Patient/327b0c02-5e31-5c1d-979b-3b3b942b5770,Encounter/e49adf4e-9a22-5063-8bda-0c464c3e3261,2156-03-23T23:30:00-04:00,2156-03-24 13:34:00,T-piece
4,f6578e25-18dc-56d7-ad93-c2345f67d100,Patient/327b0c02-5e31-5c1d-979b-3b3b942b5770,Encounter/e49adf4e-9a22-5063-8bda-0c464c3e3261,2156-03-23T19:30:00-04:00,2156-03-24 09:25:00,T-piece
...,...,...,...,...,...,...
1536865,d2009dd6-4407-52cc-b882-c97f638f5362,Patient/fff67862-3560-5403-9ae6-2f645ea1894f,Encounter/44c419df-b755-51aa-a343-d6741f824003,2148-02-19T16:00:00-05:00,2148-02-20 08:02:00,Nasal cannula
1536866,b1b41186-c491-5580-81fa-1e83044246d6,Patient/fff67862-3560-5403-9ae6-2f645ea1894f,Encounter/44c419df-b755-51aa-a343-d6741f824003,2148-02-18T08:00:00-05:00,2148-02-19 00:46:00,Nasal cannula
1536867,6bfdc969-7938-5952-be49-8a26af634277,Patient/fff67862-3560-5403-9ae6-2f645ea1894f,Encounter/44c419df-b755-51aa-a343-d6741f824003,2148-02-20T08:00:00-05:00,2148-02-20 23:08:00,Nasal cannula
1536868,26b05dc8-a2ae-5d13-a43c-ae1645818418,Patient/fff67862-3560-5403-9ae6-2f645ea1894f,Encounter/44c419df-b755-51aa-a343-d6741f824003,2148-02-20T12:00:00-05:00,2148-02-21 03:23:00,None


In [13]:
dv_oxygen_delivery_device = spark.sql("""
    SELECT patient_id AS subject_id, encounter_id AS stay_id, CAST(charttime AS TIMESTAMP) as charttime, value AS o2_delivery_device FROM rv_oxygen_delivery_device
""")
dv_oxygen_delivery_device.createOrReplaceTempView('dv_oxygen_delivery_device')
dv_oxygen_delivery_device.toPandas()

,subject_id,stay_id,charttime,o2_delivery_device
0,Patient/327b0c02-5e31-5c1d-979b-3b3b942b5770,Encounter/e49adf4e-9a22-5063-8bda-0c464c3e3261,2156-03-25 06:00:00,T-piece
1,Patient/327b0c02-5e31-5c1d-979b-3b3b942b5770,Encounter/e49adf4e-9a22-5063-8bda-0c464c3e3261,2156-03-25 00:00:00,T-piece
2,Patient/327b0c02-5e31-5c1d-979b-3b3b942b5770,Encounter/e49adf4e-9a22-5063-8bda-0c464c3e3261,2156-03-24 18:00:00,T-piece
3,Patient/327b0c02-5e31-5c1d-979b-3b3b942b5770,Encounter/e49adf4e-9a22-5063-8bda-0c464c3e3261,2156-03-24 13:30:00,T-piece
4,Patient/327b0c02-5e31-5c1d-979b-3b3b942b5770,Encounter/e49adf4e-9a22-5063-8bda-0c464c3e3261,2156-03-24 09:30:00,T-piece
...,...,...,...,...
1536865,Patient/fff67862-3560-5403-9ae6-2f645ea1894f,Encounter/44c419df-b755-51aa-a343-d6741f824003,2148-02-20 07:00:00,Nasal cannula
1536866,Patient/fff67862-3560-5403-9ae6-2f645ea1894f,Encounter/44c419df-b755-51aa-a343-d6741f824003,2148-02-18 23:00:00,Nasal cannula
1536867,Patient/fff67862-3560-5403-9ae6-2f645ea1894f,Encounter/44c419df-b755-51aa-a343-d6741f824003,2148-02-20 23:00:00,Nasal cannula
1536868,Patient/fff67862-3560-5403-9ae6-2f645ea1894f,Encounter/44c419df-b755-51aa-a343-d6741f824003,2148-02-21 03:00:00,None


In [14]:
# Alternative way to get the first intervention
stays_with_interventions = spark.sql("""
WITH odd AS (
    SELECT subject_id, stay_id, charttime, o2_delivery_device AS o2_delivery_device FROM dv_oxygen_delivery_device
), odd1 AS (SELECT *, 
    CASE
    -- tracheostomy
    WHEN o2_delivery_device IN (
        'Tracheostomy tube',
        'Trach mask ' -- 16435 observations
        -- 'T-piece', -- 1135 observations (T-piece could be either InvasiveVent or Tracheostomy)
        ) THEN 'Tracheostomy'
    -- mechanical / invasive ventilation
    WHEN o2_delivery_device IN (
        'Endotracheal tube'
        ) THEN 'InvasiveVent'
    -- NIV
    WHEN o2_delivery_device IN (
        'Bipap mask ', -- 8997 observations
        'CPAP mask ' -- 5568 observations
        ) THEN 'NonInvasiveVent'
    -- high flow nasal cannula
    when o2_delivery_device IN (
        'High flow nasal cannula' -- 925 observations
        ) THEN 'HFNC'
    -- non rebreather
    WHEN o2_delivery_device IN ( 
        'Non-rebreather', -- 5182 observations
        'Face tent', -- 24601 observations
        'Aerosol-cool', -- 24560 observations
        'Venti mask ', -- 1947 observations
        'Medium conc mask ', -- 1888 observations
        'Ultrasonic neb', -- 9 observations
        'Vapomist', -- 3 observations
        'Oxymizer', -- 1301 observations
        'High flow neb', -- 10785 observations
        'Nasal cannula'
        ) THEN 'SupplementalOxygen'
    WHEN o2_delivery_device in (
        'None'
     ) THEN 'None'
    -- not categorized: other
    ELSE NULL END AS ventilation_status
FROM odd WHERE o2_delivery_device  IS NOT NULL
), ventilation_type AS (
SELECT subject_id, stay_id, charttime AS starttime, ventilation_status FROM odd1
), iws AS (
    SELECT stay_id, starttime as  inttime, ventilation_status AS int_type,
row_number() OVER (PARTITION BY stay_id ORDER BY starttime) AS int_sequence
FROM ventilation_type
WHERE ventilation_status NOT in ('None', 'SupplementalOxygen') AND ventilation_status IS NOT NULL
)
SELECT stay_id, inttime, int_type FROM iws WHERE int_sequence = 1
""")
stays_with_interventions.count()

33376

## Observations 
### Chartevents

In [15]:
def obs_for_coding(system, code, view_name = None):
    obs_view = data.view('Observation',
                         select = [
                             {
                                 "column": [
                                     {
                                         "name": "id",
                                         "path": "id",
                                         "type": "id",
                                         "collection": False
                                     },
                                     {
                                         "name": "patient_id",
                                         "path": "subject.getReferenceKey()",
                                         "type": "string",
                                         "collection": False
                                     },
                                     {
                                         "name": "encounter_id",
                                         "path": "encounter.getReferenceKey()",
                                         "type": "string",
                                         "collection": False
                                     },
                                     {
                                         "name": "date",
                                         "path": "effectiveDateTime",
                                         "type": "dateTime",
                                         "collection": False
                                     },
                                     {
                                         "name": "value",
                                         "path": "valueQuantity.value",
                                         "type": "decimal",
                                         "collection": False
                                     },
                                     {
                                         "name": "unit",
                                         "path": "valueQuantity.unit",
                                         "collection": False
                                     }
                                 ]
                             }
                         ],
                         where= [
                             {
                                 "path":f"code.coding.system = '{system}'"
                             },
                             {
                                 "path":f"code.coding.code = '{code}'"
                             },

                         ]
                         )
    if view_name is not None:
        obs_view.createOrReplaceTempView(view_name)
    return obs_view


CS_MIMIC_CHARTEVENTS_D_ITEMS = 'http://mimic.mit.edu/fhir/mimic/CodeSystem/mimic-chartevents-d-items'
CS_MIMIC_D_LABITEMS = 'http://mimic.mit.edu/fhir/mimic/CodeSystem/mimic-d-labitems'

def obs_chartevent_for_code(code, view_name = None):
    return obs_for_coding(CS_MIMIC_CHARTEVENTS_D_ITEMS, code, view_name)

def obs_labitems_for_code(code, view_name = None):
    return obs_for_coding(CS_MIMIC_D_LABITEMS, code, view_name)

In [16]:
obs_resp_rate = obs_chartevent_for_code('224690', 'obs_resp_rate')
obs_resp_rate.show()

+--------------------+--------------------+--------------------+--------------------+---------+--------+
|                  id|          patient_id|        encounter_id|                date|    value|    unit|
+--------------------+--------------------+--------------------+--------------------+---------+--------+
|ab0ca6e1-f633-527...|Patient/327b0c02-...|Encounter/e49adf4...|2156-03-23T03:30:...| 9.000000|insp/min|
|0d13f866-69a8-560...|Patient/327b0c02-...|Encounter/e49adf4...|2156-03-23T00:00:...| 8.000000|insp/min|
|6a1da103-1acf-545...|Patient/327b0c02-...|Encounter/e49adf4...|2156-03-22T19:30:...|12.000000|insp/min|
|e7777a59-aea0-53f...|Patient/327b0c02-...|Encounter/e49adf4...|2156-03-22T04:00:...|11.000000|insp/min|
|381d9899-5498-580...|Patient/327b0c02-...|Encounter/e49adf4...|2156-03-21T23:30:...|14.000000|insp/min|
|93796af2-500d-5b0...|Patient/327b0c02-...|Encounter/e49adf4...|2156-03-21T19:45:...|20.000000|insp/min|
|6aed573d-f8b1-5ce...|Patient/327b0c02-...|Encounter/e4

In [17]:
obs_spo2 = obs_chartevent_for_code('220277', 'obs_spo2')
obs_spo2.show()

+--------------------+--------------------+--------------------+--------------------+----------+----+
|                  id|          patient_id|        encounter_id|                date|     value|unit|
+--------------------+--------------------+--------------------+--------------------+----------+----+
|33fbe9fd-9006-56d...|Patient/327b0c02-...|Encounter/e49adf4...|2156-03-17T03:00:...|100.000000|   %|
|cdc16b23-7699-531...|Patient/327b0c02-...|Encounter/e49adf4...|2156-03-17T02:00:...|100.000000|   %|
|673ffabc-6c96-554...|Patient/327b0c02-...|Encounter/e49adf4...|2156-03-17T01:00:...|100.000000|   %|
|65786cc2-39ff-543...|Patient/327b0c02-...|Encounter/e49adf4...|2156-03-16T23:00:...|100.000000|   %|
|b6aca445-3efd-570...|Patient/327b0c02-...|Encounter/e49adf4...|2156-03-16T22:00:...|100.000000|   %|
|e5cff1d5-66e5-505...|Patient/327b0c02-...|Encounter/e49adf4...|2156-03-16T21:00:...|100.000000|   %|
|1dd13aeb-366c-588...|Patient/327b0c02-...|Encounter/e49adf4...|2156-03-16T19:00:.

In [18]:
obs_heart_rate = obs_chartevent_for_code('220045', 'obs_heart_rate')
obs_heart_rate.show()

+--------------------+--------------------+--------------------+--------------------+----------+----+
|                  id|          patient_id|        encounter_id|                date|     value|unit|
+--------------------+--------------------+--------------------+--------------------+----------+----+
|2baf4fb9-6887-532...|Patient/327b0c02-...|Encounter/e49adf4...|2156-03-17T05:00:...| 80.000000| bpm|
|c113df93-2362-59f...|Patient/327b0c02-...|Encounter/e49adf4...|2156-03-17T04:00:...| 80.000000| bpm|
|b30e5e3f-382a-576...|Patient/327b0c02-...|Encounter/e49adf4...|2156-03-17T03:00:...| 78.000000| bpm|
|f2e1b838-abf5-57b...|Patient/327b0c02-...|Encounter/e49adf4...|2156-03-17T02:00:...| 80.000000| bpm|
|328010f0-f325-533...|Patient/327b0c02-...|Encounter/e49adf4...|2156-03-17T01:00:...| 68.000000| bpm|
|78b84b7e-4530-509...|Patient/327b0c02-...|Encounter/e49adf4...|2156-03-17T00:00:...| 75.000000| bpm|
|07acc617-0e3f-565...|Patient/327b0c02-...|Encounter/e49adf4...|2156-03-16T23:00:.

In [19]:
#
# alike mimiciv_derrived.vital_signs
#
derr_vital_signs_view = spark.sql("""
WITH ou AS (
SELECT patient_id, encounter_id, CAST(date AS TIMESTAMP) as obs_time, value AS resp_rate, NULL AS heart_rate, NULL AS spo2 FROM obs_resp_rate 
UNION  
SELECT patient_id, encounter_id, CAST(date AS TIMESTAMP) as obs_time, NULL AS resp_rate, value AS heart_rate,  NULL AS spo2 FROM obs_heart_rate 
UNION  
SELECT patient_id, encounter_id, CAST(date AS TIMESTAMP) as obs_time, NULL AS resp_rate, NULL AS heart_rate,  value AS spo2 FROM obs_spo2 
) 
SELECT patient_id, encounter_id, obs_time, FIRST(resp_rate) AS resp_rate, FIRST(heart_rate) AS heart_rate, FIRST(spo2) AS spo2 FROM ou 
GROUP BY patient_id, encounter_id, obs_time
""")
derr_vital_signs_view.sample(0.02).toPandas()

,patient_id,encounter_id,obs_time,resp_rate,heart_rate,spo2
0,Patient/981c9fae-f498-58fc-9c9d-e400dae485ff,Encounter/a92486e2-c0ca-58ac-a2e6-ddb2ef676aba,2185-04-15 20:00:00,28.000000,None,None
1,Patient/ee14b6e8-9a6d-5484-84d8-dee2bc795128,Encounter/74f701cd-33a8-5862-97e6-6f5c0e2a4d13,2181-11-23 11:00:00,18.000000,None,None
2,Patient/ee1dd889-5efd-5297-a3b5-19980ea145ab,Encounter/85edc39a-ef60-55aa-9cbd-82d672d94e79,2163-10-14 05:00:00,19.000000,None,None
3,Patient/dc68774d-2198-5caa-8dc5-769bb7afd090,Encounter/e4265880-8e09-5500-a3b8-ae13fbeb4426,2168-02-06 10:00:00,16.000000,None,None
4,Patient/a6e2173f-d1ef-55d1-999e-4da6be7b5f1f,Encounter/41b20ce6-142c-5565-bae3-80710b284504,2146-01-06 19:00:00,13.000000,None,None
...,...,...,...,...,...,...
138458,Patient/f642662e-6db3-56db-815c-59afc9adbeba,Encounter/9315c359-300f-5afd-bc24-696b46c2f51f,2158-11-29 01:01:00,None,None,92.000000
138459,Patient/774816bf-2874-5a1a-a342-9c1dd993ce78,Encounter/0e0cd3f5-bd08-5a7a-8ed5-cf4482d37234,2119-10-23 05:25:00,None,None,94.000000
138460,Patient/240a5e20-49fb-5924-bc9d-020bc2331f21,Encounter/1cad8c5f-0be4-5e00-a8d4-4dba8bbbf0ad,2181-04-09 05:25:00,None,None,97.000000
138461,Patient/2402a0f7-88d3-5978-8296-d01283cec2df,Encounter/2d6f5df8-6847-5291-9766-e74ecb97314c,2189-04-23 04:04:00,None,None,95.000000


### Labevents

In [20]:
# bg so2
obs_bg_so2 = obs_labitems_for_code('50817', 'obs_bg_so2')
# AND < 100
obs_bg_so2.show()

+--------------------+--------------------+--------------------+--------------------+---------+----+
|                  id|          patient_id|        encounter_id|                date|    value|unit|
+--------------------+--------------------+--------------------+--------------------+---------+----+
|a13ebf5d-d467-5a8...|Patient/b37bfbf8-...|Encounter/0130857...|2167-11-28T08:44:...|64.000000|   %|
|50cf9af5-fe15-58c...|Patient/b37bfbf8-...|Encounter/0130857...|2167-11-28T04:33:...|66.000000|   %|
|9dbc3b8c-ba80-542...|Patient/b37bfbf8-...|Encounter/0130857...|2167-11-28T15:07:...|67.000000|   %|
|bab53891-1a57-504...|Patient/b37bfbf8-...|Encounter/0130857...|2167-11-29T08:24:...|95.000000|   %|
|98be9865-9de2-56d...|Patient/b37bfbf8-...|Encounter/0130857...|2167-11-29T08:27:...|77.000000|   %|
|46d1f001-5591-566...|Patient/b37bfbf8-...|Encounter/0130857...|2167-11-28T23:56:...|76.000000|   %|
|3f44fbc5-8a05-569...|Patient/b37d3c55-...|                NULL|2142-09-27T09:22:...|20.000

In [21]:
# bg so2
obs_bg_pco2 = obs_labitems_for_code('50818', 'obs_bg_pco2')
obs_bg_pco2.show()

+--------------------+--------------------+--------------------+--------------------+---------+-----+
|                  id|          patient_id|        encounter_id|                date|    value| unit|
+--------------------+--------------------+--------------------+--------------------+---------+-----+
|4f59abea-cfc1-52a...|Patient/b37bfbf8-...|Encounter/0130857...|2167-11-28T10:49:...|29.000000|mm Hg|
|232aa8a8-e9cc-502...|Patient/b37bfbf8-...|Encounter/0130857...|2167-11-30T04:12:...|30.000000|mm Hg|
|691b5fab-8c5b-529...|Patient/b37bfbf8-...|                NULL|2167-11-28T01:35:...|32.000000|mm Hg|
|384a0bbf-6e22-5a7...|Patient/b37bfbf8-...|Encounter/0130857...|2167-11-28T08:41:...|30.000000|mm Hg|
|7e18dbaf-4d7f-53b...|Patient/b37bfbf8-...|Encounter/0130857...|2167-11-28T04:29:...|30.000000|mm Hg|
|2d6a552f-8c7f-581...|Patient/b37bfbf8-...|Encounter/0130857...|2167-12-01T05:12:...|31.000000|mm Hg|
|24e34308-14a1-573...|Patient/b37bfbf8-...|Encounter/0130857...|2167-11-29T01:45:.

In [22]:
# hemoglobin
obs_bg_hemoglobin = obs_labitems_for_code('50811', 'obs_bg_hemoglobin')
obs_bg_hemoglobin.toPandas()

,id,patient_id,encounter_id,date,value,unit
0,893744a0-2950-54e1-a5e3-9e6cd0e5a585,Patient/b37b2674-06d0-5bb7-8da0-0ece76cee89a,None,2168-02-17T12:00:00-05:00,16.600000,g/dL
1,b3981c0c-e358-5f4b-95a7-6b43f9f3049f,Patient/b37dc22c-5665-5df9-bd04-42839c5f71ae,Encounter/f66b075c-f4e4-5144-bc11-bb49e70d80a9,2182-01-17T18:27:00-05:00,10.300000,g/dL
2,9268948a-379d-54ce-98f1-1ec52dd474aa,Patient/b38088af-1a05-53bb-ab62-1a9abd65012e,Encounter/c767f541-f2ba-5095-aa1c-d5085807d9f5,2196-12-19T10:33:00-05:00,12.300000,g/dL
3,9afe3ec7-debb-501a-a31b-a727c965334e,Patient/b38088af-1a05-53bb-ab62-1a9abd65012e,Encounter/c767f541-f2ba-5095-aa1c-d5085807d9f5,2196-12-19T12:37:00-05:00,11.300000,g/dL
4,de042f86-647f-5342-804b-2f38b67d7d74,Patient/b38088af-1a05-53bb-ab62-1a9abd65012e,Encounter/c767f541-f2ba-5095-aa1c-d5085807d9f5,2196-12-19T15:18:00-05:00,11.100000,g/dL
...,...,...,...,...,...,...
111628,2fff3542-792b-5260-b07c-6f310e942d23,Patient/fff93afa-c344-5dc5-8a87-d5bc4476d2ba,Encounter/ebad3675-d064-5e61-8ff2-b16c60038efb,2117-04-01T12:10:00-04:00,6.800000,g/dL
111629,d7a374d3-a1ab-53e3-a54e-93267e0baf1e,Patient/fff93afa-c344-5dc5-8a87-d5bc4476d2ba,Encounter/ebad3675-d064-5e61-8ff2-b16c60038efb,2117-04-01T12:48:00-04:00,6.600000,g/dL
111630,abee4d7b-285b-5a66-850b-d326e6ba47f3,Patient/fff93afa-c344-5dc5-8a87-d5bc4476d2ba,Encounter/ebad3675-d064-5e61-8ff2-b16c60038efb,2117-04-01T13:03:00-04:00,6.600000,g/dL
111631,1158866f-9ce5-5181-88f5-dcd791691cbb,Patient/fff93afa-c344-5dc5-8a87-d5bc4476d2ba,Encounter/ebad3675-d064-5e61-8ff2-b16c60038efb,2117-04-01T14:19:00-04:00,7.800000,g/dL


In [23]:
#
# alike mimiciv_derrived.bg
#
derr_bg_view = spark.sql("""
WITH ou AS (
SELECT patient_id, encounter_id, CAST(date AS TIMESTAMP) as obs_time, value AS so2, NULL AS pco2, NULL AS hemoglobin  FROM obs_bg_so2 
UNION  
SELECT patient_id, encounter_id, CAST(date AS TIMESTAMP) as obs_time, NULL AS so2, value AS pco2, NULL AS hemoglobin FROM obs_bg_pco2
UNION  
SELECT patient_id, encounter_id, CAST(date AS TIMESTAMP) as obs_time, NULL AS so2, NULL AS pco2, value AS hemoglobin FROM obs_bg_hemoglobin  
) 
SELECT patient_id, encounter_id, obs_time, FIRST(so2) AS so2, FIRST(pco2) AS pco2, FIRST(hemoglobin) AS hemoglobin FROM ou 
GROUP BY patient_id, encounter_id, obs_time
""")
derr_bg_view.sample(0.02).toPandas()

,patient_id,encounter_id,obs_time,so2,pco2,hemoglobin
0,Patient/0b330ffd-8b21-51d0-b91f-9f39de481ad2,Encounter/f325ab8a-a604-585b-89d8-e5d2676c0221,2114-08-01 20:04:00,65.000000,None,None
1,Patient/29abe168-5a11-512e-b5cb-7b686188a350,Encounter/f0f6f2ab-0602-5446-9188-30d7be2d9a45,2114-06-14 01:22:00,98.000000,None,None
2,Patient/f24e8cf5-d268-5c5c-be0c-ca645224f8f6,Encounter/1ba38cb1-4d5f-5986-89df-2a3820e7b634,2151-06-07 18:01:00,98.000000,None,None
3,Patient/a15664ba-9f04-5223-ad3e-33c968abf70b,Encounter/42252eb0-d954-5202-b3d1-0b11c9158d0a,2145-06-20 12:33:00,50.000000,None,None
4,Patient/d10c8c20-3658-53c6-8594-a2e1afafaa3e,Encounter/4e53cf64-63f7-51f7-b644-c5fefc89d3d2,2148-10-10 22:17:00,93.000000,None,None
...,...,...,...,...,...,...
11252,Patient/d9f25663-540e-5db7-97be-9782741944fe,None,2114-05-15 14:24:00,None,None,8.500000
11253,Patient/850346fa-4499-5c93-846c-b628e0d80367,Encounter/03f35944-3982-59e2-a898-9735dab87ed0,2147-11-09 12:33:00,None,None,8.100000
11254,Patient/1efcb51a-2f45-5622-a2ec-b11714ef9591,Encounter/c1ef6e15-f593-515c-b923-ce28d093a25a,2185-12-30 16:07:00,None,None,7.400000
11255,Patient/5b43140c-d1b3-552a-a5e2-422e434ac595,None,2190-11-22 11:50:00,None,None,14.600000


## Encounters (ICU) 